In [21]:
import pandas as pd
import numpy as np
import os


In [22]:
## Get address of where the repository is located on your computer.

# Will raise exception is you're in the wrong folder.
if not os.getcwd().endswith("02443-SimSolarEnergy\\code"):
    raise Exception("Change working directory to the code folder.")
repo_loc = os.getcwd()[ : -len("\\code")]


In [23]:
## Print all the files we need

# List for file names, empty dictionary for data
file_names = []
all_data = {}

# Loop through file names
for upper_file_name in os.listdir(repo_loc + "\data"):
    if upper_file_name.startswith("UK"):
        loc = repo_loc + "\\data\\" + upper_file_name
        for file_name in os.listdir(loc):
            name = file_name[:-len(".csv")] + "_" + upper_file_name[-2:]
            all_data[name] = pd.read_csv(loc + "\\" + file_name)

# Print data
for name, data in all_data.items():
    print(name.upper() + "\n" + "-" * len(name))
    print(data.head())
    print("\n\n")


C:\Users\Jan Warhuus\.virtualenvs\02443-SimSolarEnergy-Fn5Vr-Iv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (131,139,181,191,196,216,221,269,275,334,339,344,349,376,502,507,512,517,522,542,563,566,569,572) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


DAT_14
------
     serial  pnum               t_start  eptime  whatdoing  WhereWhen
0  11011202     1  2014-12-11T03:00:00Z     110        110         11
1  11011202     1  2014-12-11T04:50:00Z      10       8219         11
2  11011202     1  2014-12-11T05:00:00Z      10        310         11
3  11011202     1  2014-12-11T05:10:00Z      10       3210         11
4  11011202     1  2014-12-11T05:20:00Z      10       3110         11



DAT_HOUSEHOLD_14
----------------
     serial  num_adult  num_child  num_room  microwav  dishwash  dhhtype
0  11010903          2          0         8      True      True      3.0
1  11010904          2          0         6     False      True      3.0
2  11010906          3          0         4      True     False      7.0
3  11010907          2          1         3      True     False      2.0
4  11010908          1          0         6      True      True      1.0



DAT_INDI_14
-----------
     serial  strata  psu  pnum  DMFlag  HhOut  IndOut  ind_wt  I

In [24]:
# Looks like the files in 15 and 16 are the same...? And they maybe come
# from the ones in 13? Let's ook at the sahape of the frames to confirm.

for name, data in all_data.items():
    print(name.upper() + "\n" + "-" * len(name))
    print(data.shape)
    print("\n\n")
    

DAT_14
------
(587632, 6)



DAT_HOUSEHOLD_14
----------------
(4733, 7)



DAT_INDI_14
-----------
(11421, 603)



ACTIVITY_15
-----------
(587632, 6)



ACTIVITY_SHORT_15
-----------------
(500, 6)



HOUSEHOLD_15
------------
(4733, 7)



INDIVIDUAL_15
-------------
(11421, 5)



ACTIVITY_SHORT_16
-----------------
(500, 7)



HOUSEHOLD_16
------------
(4733, 7)



INDIVIDUAL_16
-------------
(11421, 5)





In [44]:
# So a few a few of the dataframes do look exactly the same:
# 1. household_16, household_15 and dat_household_14 are the same
# 2. individual_15 and individual_16 look the same
# 3. activity_15 and dat_14 look the same

# So now I'll just check each column to see if there are any discrepencies
# between the dataframes just mentioned. I'll start with point 1:

# POINT 1
print((all_data["household_16"] == all_data["household_15"]).all(), "\n")
print((all_data["household_16"] == all_data["dat_household_14"]).all(), "\n")

serial        True
num_adult     True
num_child     True
num_room      True
microwav      True
dishwash      True
dhhtype      False
dtype: bool 

serial        True
num_adult     True
num_child     True
num_room      True
microwav      True
dishwash      True
dhhtype      False
dtype: bool 



In [65]:
# Ok so only the dhhtype are different... where are they different?
(all_data["household_16"].dhhtype == all_data["household_15"].dhhtype).value_counts()

True     4610
False     123
Name: dhhtype, dtype: int64

In [55]:
(all_data["household_16"].dhhtype == all_data["dat_household_14"].dhhtype).value_counts()

False    4610
True      123
Name: dhhtype, dtype: int64

In [58]:
# Ok so most entries match for both cases. Doesn't really make
# a difference in terms of the model if we pick one over the other.

# POINT 2
print((all_data["individual_16"] == all_data["individual_15"]).all(), "\n")

serial       True
pnum         True
sex          True
age          True
deconact    False
dtype: bool 



In [64]:
# Also the same except for this deconact thing
(all_data["individual_16"].deconact == all_data["individual_15"].deconact).value_counts()

True     11396
False       25
Name: deconact, dtype: int64

In [61]:
# Ok so most of them match.

# POINT 3
print((all_data["activity_15"] == all_data["dat_14"]).all())

serial        True
pnum          True
t_start      False
eptime        True
whatdoing    False
WhereWhen     True
dtype: bool


In [62]:
# Ok so t_start and whatdoing are different.
(all_data["activity_15"].whatdoing != all_data["dat_14"].whatdoing).value_counts()

True     587584
False        48
Name: whatdoing, dtype: int64

In [63]:
(all_data["activity_15"].t_start != all_data["dat_14"].t_start).value_counts()

False    585569
True       2063
Name: t_start, dtype: int64

In [ ]:
# Ok very weird... so most of the start times are exactly the same between
# the two dataframes. The whatdoing are different though, because whatdoing
# in 15 has the short codes (eg. 0, 8, etc.) while whatdoing in 14 has the
# long codes (11, 3945, etc.). So I'll use the data from 14 I suppose.